In [1]:
import pandas as pd

## Data cleaning

## Only select winter season data

In [2]:
df_rawdata1 = pd.read_csv('Chinese Thermal Comfort Dataset Class I.csv', encoding='GBK')
df_rawdata2 = pd.read_csv('Chinese Thermal Comfort Dataset Class II.csv', encoding='GBK')
df_rawdata3 = pd.read_csv('Chinese Thermal Comfort Dataset Class III.csv', encoding='GBK')
# concatenate the three datasets
df_rawdata = pd.concat([df_rawdata1, df_rawdata2, df_rawdata3], ignore_index=True)
print("total records:", len(df_rawdata))
# only select A4.Season = heating season
df_heating = df_rawdata[df_rawdata['A4.Season'] == 'Winter Season']
print("total heating records:", len(df_heating))

total records: 41977
total heating records: 15992


## Clean city names

In [3]:
df_heating['A5.City'].unique()

array(['Hebei Province,Handan City', 'Henan Province,Zhengzhou City',
       'Henan Province,Luoyang City', 'Tianjin City',
       'Liaoning Province,Dalian City',
       'Heilongjiang Province, Harbin City', 'Beijing',
       'Jiangsu Province,Changzhou City', 'Shanghai City',
       "Shaanxi Province,Xi'an City", 'Guangdong Province,Guangzhou City',
       'Beijing City', 'Baotou City', 'Turpan',
       'Hubei Province, Wuhan City', "Shaanxi Province, Xi'an City",
       'Hunan Province, Changsha City', 'Yunnan Province,Kunming City',
       'Lhasa City', 'Henan Province,Jiaozuo City',
       'Sichuan Province, Zigong City', 'Liaoning Province, Benxi City',
       'Henan Province, Nanyang City', 'Liaoning Province, Dalian City',
       'Hainan Province, Sanya City', 'Hunan Province, Hengyang City',
       'Shanxi Province, Lvliangfenyang City',
       'Henan Province, Pingdingshan City',
       'Yunnan Province, Kunming City', 'Shandong Province, Qingdao City',
       'Fujian Provinc

### Organizing city names in the same format (without comma and space)

In [4]:
df_heating = df_heating.copy()
# remove comma and replace space by _ in the A5.City strings
df_heating['A5.City'] = df_heating['A5.City'].str.replace(' ', '')
df_heating['A5.City'] = df_heating['A5.City'].str.replace(',', '')
# replace BeijingCity by Beijing
df_heating['A5.City'] = df_heating['A5.City'].str.replace('BeijingCity', 'Beijing')

In [5]:
df_heating['A5.City'].unique()

array(['HebeiProvinceHandanCity', 'HenanProvinceZhengzhouCity',
       'HenanProvinceLuoyangCity', 'TianjinCity',
       'LiaoningProvinceDalianCity', 'HeilongjiangProvinceHarbinCity',
       'Beijing', 'JiangsuProvinceChangzhouCity', 'ShanghaiCity',
       "ShaanxiProvinceXi'anCity", 'GuangdongProvinceGuangzhouCity',
       'BaotouCity', 'Turpan', 'HubeiProvinceWuhanCity',
       'HunanProvinceChangshaCity', 'YunnanProvinceKunmingCity',
       'LhasaCity', 'HenanProvinceJiaozuoCity',
       'SichuanProvinceZigongCity', 'LiaoningProvinceBenxiCity',
       'HenanProvinceNanyangCity', 'HainanProvinceSanyaCity',
       'HunanProvinceHengyangCity', 'ShanxiProvinceLvliangfenyangCity',
       'HenanProvincePingdingshanCity', 'ShandongProvinceQingdaoCity',
       'FujianProvinceZhenjiangCity', 'ShandongProvinceYantaiCity',
       'HohhotCity', 'FujianProvinceFuzhouCity',
       'ShandongProvinceXintaiCity', 'HenanProvinceKaifengCity',
       'FujianProvinceNanpingCity'], dtype=object)

## Add latitude, outdoor temperature (January), and HDD data for each city

In [6]:
city_dict_lat = {
    "Beijing": 39.90,
    "TianjinCity": 39.12, 
    "BaotouCity": 40.65,
    "HohhotCity": 40.82,
    "LiaoningProvinceDalianCity": 38.92,
    "ShandongProvinceYantaiCity": 37.53,
    "ShanxiProvinceLvliangfenyangCity": 37.51, 
    "HebeiProvinceHandanCity": 36.62,
    "HeilongjiangProvinceHarbinCity": 45.75,
    "LiaoningProvinceBenxiCity": 41.29,

    "ShandongProvinceQingdaoCity": 36.07,
    "ShandongProvinceXintaiCity": 35.90, 

    "HenanProvinceJiaozuoCity": 35.24,
    "HenanProvinceKaifengCity": 34.79,
    "HenanProvinceZhengzhouCity": 34.75,
    "HenanProvinceLuoyangCity": 34.62,
    "HenanProvincePingdingshanCity": 33.74,
    "HenanProvinceNanyangCity": 32.99,

    "ShaanxiProvinceXi'anCity": 34.26,

    "JiangsuProvinceChangzhouCity": 31.78,  
    "ShanghaiCity": 31.23,
    "Turpan": 42.94,
    "LhasaCity": 29.65, 
    "HubeiProvinceWuhanCity": 30.58,  
    "SichuanProvinceZigongCity": 29.34,
    "HunanProvinceChangshaCity": 28.23,
    "HunanProvinceHengyangCity": 26.89,

    "FujianProvinceNanpingCity": 26.64,  
    "FujianProvinceFuzhouCity": 26.08,
    "FujianProvinceZhenjiangCity": 26.10,

    "YunnanProvinceKunmingCity": 24.87, 
    "GuangdongProvinceGuangzhouCity": 23.13,  
    "HainanProvinceSanyaCity": 18.25,
}
# create a new column 'latitude' and in df_heating and assign the latitude value to corresponding city
df_heating['latitude'] = df_heating['A5.City'].map(city_dict_lat)

In [7]:
# mannual find the minimum monthly outdoor temperature for each city
city_t_out_min = {
    "HebeiProvinceHandanCity": -0.8,  
    "HenanProvinceZhengzhouCity": 0.8,
    "HenanProvinceKaifengCity": 0.6,
    "HenanProvinceJiaozuoCity": 1.5, 
    "HenanProvinceNanyangCity": 1.8, 
    "HenanProvincePingdingshanCity": 0.7,  # Assume same as Xuchang
    "HenanProvinceLuoyangCity": 0.8,  # Assume same as Zhengzhou

    "ShaanxiProvinceXi'anCity": 3.1,
    "ShanxiProvinceLvliangfenyangCity": -4.8,  # Assume same as Taiyuan

    "Beijing": -2.8,  
    "TianjinCity": -3.3,  
    "LiaoningProvinceDalianCity": -3.4, 
    "LiaoningProvinceBenxiCity": -10.9,
    "HeilongjiangProvinceHarbinCity": -17.4,  

    "JiangsuProvinceChangzhouCity": 3.9,  # Assume same as Wuxi
    "ShanghaiCity": 5.2,  

    "BaotouCity": -10.4,  # Estimated
    "HohhotCity": -10.7, 
    "LhasaCity": -0.2,  
    "Turpan": -6.5,  

    "HubeiProvinceWuhanCity": 4.1,  
    "HunanProvinceChangshaCity": 5.2,  
    "HunanProvinceHengyangCity": 6.3,  
 
    "SichuanProvinceZigongCity": 7.6,  
    "YunnanProvinceKunmingCity": 9.2,
    "GuangdongProvinceGuangzhouCity": 13.7,  
    "HainanProvinceSanyaCity": 21,  

    "ShandongProvinceQingdaoCity": 0.1,  
    "ShandongProvinceYantaiCity": -1.3,  # Assume same as Longkou
    "ShandongProvinceXintaiCity": -7.4,  # Assume same as Taishan

    "FujianProvinceZhenjiangCity": 10,  # Typo, but should be a Fujian city, assum same as Sanming
    "FujianProvinceNanpingCity": 10.1,  # Assume same as Sanming
    "FujianProvinceFuzhouCity": 11.3

}
# create a new column 't_out_min' and in df_heating and assign the value to corresponding city
df_heating['t_out_min'] = df_heating['A5.City'].map(city_t_out_min)

In [8]:
# mannual find the heating degree days in January for each city, PPL
city_HDD = {
    "HebeiProvinceHandanCity": 2096.8, # Xingtai weather data

    "HenanProvinceZhengzhouCity": 1886.6,
    "HenanProvinceKaifengCity": 1886.6, # Zhengzhou weather data
    "HenanProvinceJiaozuoCity": 1886.6, # Zhengzhou weather data
    "HenanProvinceNanyangCity": 1769.1, 
    "HenanProvincePingdingshanCity": 1910.3,  # Zhoukou/xihua weather data
    "HenanProvinceLuoyangCity": 1877.9, 

    "ShaanxiProvinceXi'anCity": 1901.9,
    "ShanxiProvinceLvliangfenyangCity": 3029.4,  # Jiaokou/Lishi weather data

    "Beijing": 2466,  
    "TianjinCity": 2455.6,

    "LiaoningProvinceDalianCity": 2749.2, 
    "LiaoningProvinceBenxiCity": 4920.4,
    "HeilongjiangProvinceHarbinCity": 3584.9,  

    "JiangsuProvinceChangzhouCity": 1566.2,  # Xuzhou weather data
    "ShanghaiCity": 1341.2, # Hongqiao weather data  

    "BaotouCity": 4108.3,  # Hohhot weather data
    "HohhotCity": 4108.3, 
    "LhasaCity": 2861.5,  
    "Turpan": 2482.8,  

    "HubeiProvinceWuhanCity": 1412.7,  
    "HunanProvinceChangshaCity": 1319,  
    "HunanProvinceHengyangCity": 2316.8,  
 
    "SichuanProvinceZigongCity": 1087.9, #Neijing weather data
    "YunnanProvinceKunmingCity": 1242.3,
    "GuangdongProvinceGuangzhouCity": 325.1,  
    "HainanProvinceSanyaCity": 7.1,  

    "ShandongProvinceQingdaoCity": 2409.1,  
    "ShandongProvinceYantaiCity": 2287.5,  # Longkou weather data
    "ShandongProvinceXintaiCity": 3931.3,  # Assume same as Taishan

    "FujianProvinceZhenjiangCity": 835.5,  # Typo, but should be a Fujian city, assum Sanming/Maoping
    "FujianProvinceNanpingCity": 757.8,
    "FujianProvinceFuzhouCity": 655.5

}
# create a new column
df_heating['HDD'] = df_heating['A5.City'].map(city_HDD)

In [9]:
# remove "City" from the city names
df_heating['A5.City'] = df_heating['A5.City'].str.replace('City', '')
# remove "Province" from the city names
df_heating['A5.City'] = df_heating['A5.City'].str.replace('Province', '')
# check the unique city names
df_heating['A5.City'].unique()

array(['HebeiHandan', 'HenanZhengzhou', 'HenanLuoyang', 'Tianjin',
       'LiaoningDalian', 'HeilongjiangHarbin', 'Beijing',
       'JiangsuChangzhou', 'Shanghai', "ShaanxiXi'an",
       'GuangdongGuangzhou', 'Baotou', 'Turpan', 'HubeiWuhan',
       'HunanChangsha', 'YunnanKunming', 'Lhasa', 'HenanJiaozuo',
       'SichuanZigong', 'LiaoningBenxi', 'HenanNanyang', 'HainanSanya',
       'HunanHengyang', 'ShanxiLvliangfenyang', 'HenanPingdingshan',
       'ShandongQingdao', 'FujianZhenjiang', 'ShandongYantai', 'Hohhot',
       'FujianFuzhou', 'ShandongXintai', 'HenanKaifeng', 'FujianNanping'],
      dtype=object)

## Use ASHRAE Thermal Comfort Database2 naming conventions
https://github.com/ruijis/ashrae-db-II

cooling_type -> heating_type


In [10]:
df_heating.columns

Index(['ID', 'A1.Code', 'A2.Date', 'A3.Data Contributor', 'A4.Season',
       'A5.City', 'A6.Climate Zone', 'B1.Building Type',
       'B2.Building Function', 'B3.Floors', 'B4.Building Operation Mode',
       'B5.Room (Length×Width)', 'B5.Room Height (m)', 'C1.Sex', 'C2.Age',
       'C3.Height（cm）', 'C4.Weight（kg）', 'C5.Living Years', 'D1.TSV', 'D2.TCV',
       'D3.TAV', 'D5.Clothing Insulation (clo)', 'D6.Metabolic Rate (met)',
       'Measured Height (m)', 'E1.Indoor Air Temperature (℃)',
       'E2.Indoor Relative Humidity (%)', 'E3.Indoor Air Velocity (m/s)',
       'E4.Globe Temperature (℃)', 'E5.Roof Temperature (℃)',
       'E5.Wall Temperature (℃)', 'E5.Floor Temperature (℃)',
       'F1.Operative Temperature (℃)', 'F2.Mean Radiant Temperature (℃)',
       'F3.Radiant Temperature Asymmetry (℃)', 'F4.PMV', 'F5.PPD',
       'G1.Real-Time Outdoor Temperature (℃)',
       'G2.Mean Daily Outdoor Temperature (℃)',
       'G3.Monthly Mean Outdoor Temperature (℃)',
       'G4.7-Day Run

In [11]:
# rename general information
df_heating = df_heating.rename(columns={'ID': 'suject_id'})
df_heating = df_heating.rename(columns={'A2.Date': 'timestamp'})
df_heating = df_heating.rename(columns={'A3.Data Contributor': 'contributor'})
df_heating = df_heating.rename(columns={'A4.Season': 'season'})
df_heating = df_heating.rename(columns={'A5.City': 'city'})
df_heating = df_heating.rename(columns={'A6.Climate Zone': 'climate'})
df_heating = df_heating.rename(columns={'B1.Building Type': 'building_type'})
df_heating = df_heating.rename(columns={'B4.Building Operation Mode': 'heating_type'})
# rename personal conditions
df_heating = df_heating.rename(columns={'C1.Sex': 'gender'})
df_heating = df_heating.rename(columns={'C2.Age': 'age'})
df_heating = df_heating.rename(columns={'C3.Height（cm）': 'ht'})
df_heating = df_heating.rename(columns={'C4.Weight（kg）': 'wt'})
df_heating = df_heating.rename(columns={'D1.TSV': 'thermal_sensation'})
df_heating = df_heating.rename(columns={'D2.TCV': 'thermal_comfort'})
df_heating = df_heating.rename(columns={'D3.TAV': 'thermal_acceptability'})
df_heating = df_heating.rename(columns={'D5.Clothing Insulation (clo)': 'clo'})
df_heating = df_heating.rename(columns={'D6.Metabolic Rate (met)': 'met'})
# rename  indoor environment parameters
df_heating = df_heating.rename(columns={'E1.Indoor Air Temperature (℃)': 'ta'})
df_heating = df_heating.rename(columns={'E2.Indoor Relative Humidity (%)': 'rh'})
df_heating = df_heating.rename(columns={'E3.Indoor Air Velocity (m/s)': 'vel'})
df_heating = df_heating.rename(columns={'E4.Globe Temperature (℃)': 'tg'})
df_heating = df_heating.rename(columns={'F1.Operative Temperature (℃)': 'top'})
df_heating = df_heating.rename(columns={'F2.Mean Radiant Temperature (℃)': 'tr'})
# rename outdoor temperature parameters
df_heating = df_heating.rename(columns={'G1.Real-Time Outdoor Temperature (℃)': 't_out_realtime'})
df_heating = df_heating.rename(columns={'G2.Mean Daily Outdoor Temperature (℃)': 't_out_daily'})
df_heating = df_heating.rename(columns={'G3.Monthly Mean Outdoor Temperature (℃)': 't_out_monthly'})
df_heating = df_heating.rename(columns={'G4.7-Day Running Mean Outdoor Temperature (℃)': 't_out_7'})
df_heating = df_heating.rename(columns={'G4.15-Day Running Mean Outdoor Temperature (℃)': 't_out_15'})

# rename
df_heating = df_heating.rename(columns={'G5.Mean Daily Outdoor Relative Humidity (%)': 'rh_out'})


## Check the datatypes

In [12]:
# select a sbuset of columns
df_heating_export = df_heating[['suject_id', 
                            'timestamp', 
                            'season', 
                            'city', 
                            'climate', 
                            'building_type', 
                            'heating_type', 
                            'gender', 
                            'age', 
                            'ht', 
                            'wt', 
                            'thermal_sensation', 
                            'thermal_comfort', 
                            'thermal_acceptability', 
                            'clo', 
                            'met', 
                            'ta', 
                            'rh', 
                            'vel', 
                            'tg', 
                            'top', 
                            'tr', 
                            't_out_min', 
                            't_out_monthly', 
                            'latitude',
                            'HDD']]
print(df_heating_export.shape)
df_heating_export.dtypes

(15992, 26)


suject_id                  int64
timestamp                 object
season                    object
city                      object
climate                   object
building_type             object
heating_type              object
gender                    object
age                       object
ht                       float64
wt                       float64
thermal_sensation        float64
thermal_comfort          float64
thermal_acceptability    float64
clo                      float64
met                      float64
ta                       float64
rh                         int64
vel                      float64
tg                       float64
top                      float64
tr                       float64
t_out_min                float64
t_out_monthly            float64
latitude                 float64
HDD                      float64
dtype: object

## Export CSV

In [13]:
df_heating_export.to_csv('CN_DB.csv', index=False)